<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Hebbian_Transformer_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install cryptography
import os, json, base64
from dataclasses import dataclass
from cryptography.hazmat.primitives.asymmetric import x25519, ed25519
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives import hashes, serialization

VERSION = b"\x01"
INFO = b"E2EE-v1"  # protocol label

def raw_pub_bytes(pubkey):
    return pubkey.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw,
    )

@dataclass
class Identity:
    # Long-term static identity: one X25519 (ECDH) and one Ed25519 (signing)
    x_priv: x25519.X25519PrivateKey
    e_priv: ed25519.Ed25519PrivateKey

    @property
    def x_pub(self):
        return self.x_priv.public_key()

    @property
    def e_pub(self):
        return self.e_priv.public_key()

    def public_bundle(self):
        return {
            "x25519_pub": raw_pub_bytes(self.x_pub),
            "ed25519_pub": raw_pub_bytes(self.e_pub),
        }

def generate_identity():
    return Identity(
        x_priv=x25519.X25519PrivateKey.generate(),
        e_priv=ed25519.Ed25519PrivateKey.generate(),
    )

def hkdf_derive(shared_secret: bytes, salt: bytes, info: bytes = INFO, length: int = 32):
    return HKDF(
        algorithm=hashes.SHA256(),
        length=length,
        salt=salt,
        info=info,
    ).derive(shared_secret)

def aead_encrypt(key: bytes, nonce: bytes, plaintext: bytes, aad: bytes) -> bytes:
    return ChaCha20Poly1305(key).encrypt(nonce, plaintext, aad)

def aead_decrypt(key: bytes, nonce: bytes, ciphertext: bytes, aad: bytes) -> bytes:
    return ChaCha20Poly1305(key).decrypt(nonce, ciphertext, aad)

def to_b64(b: bytes) -> str:
    return base64.b64encode(b).decode("ascii")

def from_b64(s: str) -> bytes:
    return base64.b64decode(s.encode("ascii"))

def encrypt_message(plaintext: bytes, context: bytes, sender: Identity, recipient_x25519_pub: bytes):
    # 1) Sender creates ephemeral X25519 keypair (forward secrecy)
    e_priv = x25519.X25519PrivateKey.generate()
    e_pub_bytes = raw_pub_bytes(e_priv.public_key())

    # 2) ECDH with recipient static X25519 pub
    r_x_pub = x25519.X25519PublicKey.from_public_bytes(recipient_x25519_pub)
    shared = e_priv.exchange(r_x_pub)

    # 3) Derive AEAD key via HKDF; salt binds ephemeral and recipient identity
    salt = e_pub_bytes + recipient_x25519_pub
    key = hkdf_derive(shared, salt=salt, info=INFO, length=32)

    # 4) AEAD nonce + AAD
    nonce = os.urandom(12)
    sender_ed_pub = raw_pub_bytes(sender.e_pub)
    aad = b"|".join([INFO, context, sender_ed_pub, recipient_x25519_pub])

    # 5) Encrypt
    ciphertext = aead_encrypt(key, nonce, plaintext, aad)

    # 6) Build header and sign header||ciphertext with sender’s Ed25519
    header = VERSION + sender_ed_pub + e_pub_bytes + nonce
    signature = sender.e_priv.sign(header + ciphertext)

    # 7) Emit transport-friendly JSON (base64-encoded binary fields)
    msg = {
        "version": 1,
        "sender_ed25519_pub": to_b64(sender_ed_pub),
        "sender_eph_x25519_pub": to_b64(e_pub_bytes),
        "nonce": to_b64(nonce),
        "ciphertext": to_b64(ciphertext),
        "signature": to_b64(signature),
    }
    return json.dumps(msg).encode("utf-8")

def decrypt_message(message_json: bytes, context: bytes, recipient: Identity, expected_sender_ed25519_pub: bytes):
    msg = json.loads(message_json.decode("utf-8"))

    # 1) Parse
    assert msg["version"] == 1
    sender_ed_pub_bytes = from_b64(msg["sender_ed25519_pub"])
    eph_x25519_pub_bytes = from_b64(msg["sender_eph_x25519_pub"])
    nonce = from_b64(msg["nonce"])
    ciphertext = from_b64(msg["ciphertext"])
    signature = from_b64(msg["signature"])

    # 2) Enforce trusted sender (bind to known identity)
    if sender_ed_pub_bytes != expected_sender_ed25519_pub:
        raise ValueError("Sender Ed25519 public key mismatch (untrusted sender)")

    # 3) Verify signature over header||ciphertext
    header = VERSION + sender_ed_pub_bytes + eph_x25519_pub_bytes + nonce
    ed_pub = ed25519.Ed25519PublicKey.from_public_bytes(sender_ed_pub_bytes)
    ed_pub.verify(signature, header + ciphertext)  # raises on failure

    # 4) Derive AEAD key via ECDH (recipient static × sender ephemeral)
    s_eph_pub = x25519.X25519PublicKey.from_public_bytes(eph_x25519_pub_bytes)
    shared = recipient.x_priv.exchange(s_eph_pub)
    salt = eph_x25519_pub_bytes + raw_pub_bytes(recipient.x_pub)
    key = hkdf_derive(shared, salt=salt, info=INFO, length=32)

    # 5) Build same AAD and decrypt
    aad = b"|".join([INFO, context, sender_ed_pub_bytes, raw_pub_bytes(recipient.x_pub)])
    plaintext = aead_decrypt(key, nonce, ciphertext, aad)
    return plaintext

# Identities (done once, then store securely)
alice = generate_identity()  # sender
bob   = generate_identity()  # recipient

# Public keys exchanged out-of-band (QR code, pinned fingerprint, etc.)
bob_pub_x25519 = raw_pub_bytes(bob.x_pub)
alice_pub_ed25519 = raw_pub_bytes(alice.e_pub)

# Context binds the ciphertext to an app/channel/session
context = b"my-app:dm:alice->bob"

# Alice encrypts for Bob
msg = encrypt_message(b"hello bob, only you can read this", context, alice, bob_pub_x25519)

# Bob decrypts (and authenticates Alice)
pt = decrypt_message(msg, context, bob, expected_sender_ed25519_pub=alice_pub_ed25519)
print(pt)  # b'hello bob, only you can read this'